In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 45)
pd.set_option('display.max_rows', 50)

item_master = pd.read_excel('../data/inzi/Item Master_08_18_2021 0317.xls', header=1)
material_history = pd.read_excel('../data/inzi/Material Movement History_08_19_2021 1026.xls', header=1)

material_history['Material'].value_counts()

temp_item_master = item_master[['Material', 'Material Type', 'Procurement']]

temp_item_master.loc[temp_item_master['Material'] == 'BN61-08214A']

step_one = pd.merge(material_history, temp_item_master, on="Material", how="left")


# step_one['Input'] = step_one['Quantity'] if step_one['Quantity'] > 0 else None
# step_one['Output'] = -step_one['Quantity'] if step_one['Quantity'] < 0 else None

def create_quantity(columns):
    if columns > 0:
        return [columns, 0]
    else:
        return [0, -columns]

step_one['Input'] = step_one['Quantity']
step_one['Output'] = step_one['Quantity']

# step_one['Input'] = [value for value in step_one['Input'] if step_one['Input'].all() > 0]

step_one.rename(columns={
    'Reference': 'Order Category',
    'Unnamed: 13': 'Order Data',
    'Unnamed: 14': 'Master Category',
    'Unnamed: 15': 'Master Data',
    'Unnamed: 16': 'Remark'
}, inplace=True)

step_one = step_one.iloc[1: , :]


# step_one.loc[step_one['Input'] > 0]
def temp_filter(values):
    return values if values > 0 else 0

# def filter_inout(row):
#     return row['Quantity'] if row['Quantity'] > 0 else 0

# step_one['input'] = step_one.apply(filter_inout, axis=1)

# pd.set_option('mode.chained_assignment', None)

step_one.loc[step_one['Quantity'] > 0, 'Input'] = step_one['Quantity']
step_one.loc[step_one['Quantity'] > 0, 'Output'] = 0
step_one.loc[step_one['Quantity'] <= 0, 'Input'] = 0
step_one.loc[step_one['Quantity'] <= 0, 'Output'] = -step_one['Quantity']

cols = step_one.columns.tolist()

cols = cols[:10] + [cols[-2], cols[-1]] + cols[10:-2]

step_one = step_one[cols]

step_one['Account code'] = [int(x) for x in (step_one['Movement Type'].str[:3])]

cols = step_one.columns.tolist()

cols = cols[:7] + [cols[-1]] + cols[7: -1]

step_one = step_one[cols]

summary = step_one.pivot(index=["Unnamed: 0" ,"Material"], columns="Account code", values="Quantity")
summary.reset_index(inplace=True)
del summary['Unnamed: 0']
aggregation_functions = {}
columns_order = [101, 102, 103, 321, 323, 327, 343, 344, 401, 602, 701,
                     720, 801, 809, 201, 261, 555, 601, 609, 702, 712, 721, 803]
for column in summary.columns:
    if isinstance(column, int):
        aggregation_functions[column] = 'sum'
        if column in [321, 323, 327, 343, 344, 401]:
            aggregation_functions[column] = 'first'

final_col = []
for col in columns_order:
    if col in summary.columns:
        final_col.append(col)

summary = summary.groupby(summary['Material']).aggregate(
        aggregation_functions)

for column in columns_order:
    if column not in summary.columns:
        summary[column] = np.nan
summary.fillna(0, inplace=True)
summary = summary[columns_order]
summary['IN'] = summary[[101, 602, 720, 801, 809]].sum(axis=1)
summary['OUT'] = summary[[102, 201, 261,
                            555, 601, 609, 721, 803]].sum(axis=1)
summary.reset_index(inplace=True)
summary.index.name = None

In [3]:
summary

Account code,Material,101,102,103,321,323,327,343,344,401,602,701,720,801,809,201,261,555,601,609,702,712,721,803,IN,OUT
0,0103-010237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300000.0,0.0,0.0,0.0,0.0,0.0,0.0,-483120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-483120.0
1,0103-011284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250000.0,0.0,0.0,0.0,0.0,0.0,0.0,-1208405.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1208405.5
2,0103-011475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0,0.0,0.0,0.0,0.0,0.0,0.0,-585578.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-585578.4
3,0103-011580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225000.0,0.0,0.0,0.0,0.0,0.0,0.0,-852816.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-852816.0
4,0103-011600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0,0.0,0.0,0.0,0.0,0.0,0.0,-439560.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-439560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,IZ64-04515B,1467.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1467.0,0.0
305,IZ64-04516A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-457.0
306,IZ64-04519A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-241.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-241.0
307,MD63-00103A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,-5966.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5966.0


In [ ]:
columns_name = {
    'H': 'Account code',
    'p': 'Order Category',
    'W': 'Material Type',
    'X': 'Procurement'
}

nhap_vao = [None] * 7
nhap_vao[0] = ((step_one['Account code'] == 101) & (step_one['Order Category'] == 'Purchase Order'))
nhap_vao[1] = (step_one['Account code'] == 602)
nhap_vao[2] = (step_one['Account code'] == 623)
nhap_vao[3] = (step_one['Account code'] == 701)
nhap_vao[4] = (step_one['Account code'] == 720)
nhap_vao[5] = (step_one['Account code'] == 801)
nhap_vao[6]= (step_one['Account code'] == 809)

xuat_ra = [None] * 9
xuat_ra[0] = ((step_one['Account code'] == 102) & (step_one['Order Category'] == 'Purchase Order'))
xuat_ra[1] = (step_one['Account code'] == 201)
xuat_ra[2] = (step_one['Account code'] == 261)
xuat_ra[3] = (step_one['Account code'] == 601)
xuat_ra[4] = (step_one['Account code'] == 609)
xuat_ra[5] = (step_one['Account code'] == 702)
xuat_ra[6] = (step_one['Account code'] == 712)
xuat_ra[7] = (step_one['Account code'] == 721)
xuat_ra[8] = (step_one['Account code'] == 803)

thuyen_chuyen = [None] * 2
thuyen_chuyen[0] = (step_one['Account code'].between(300, 400))
thuyen_chuyen[1] = (step_one['Account code'] == 401)

In [ ]:
# for i in range(len(nhap_vao)):
#     nhap_vao[i] = step_one.loc[nhap_vao[i]]
#     nhap_vao[i]['Tm_Rank'] = nhap_vao[i]['Material Type'].map(material_order_index)
#     nhap_vao[i].sort_values(['Tm_Rank', 'Procurement'], inplace = True)
#     nhap_vao[i].drop('Tm_Rank', 1, inplace = True)

In [ ]:
def condition_to_data(arr):
    material_type_order = ['ROH', 'HAWA', 'HALB', 'FERT']
    material_order_index = dict(zip(material_type_order, range(len(material_type_order))))
    for i in range(len(arr)):
        arr[i] = step_one.loc[arr[i]]
        arr[i]['Tm_Rank'] = arr[i]['Material Type'].map(material_order_index)
        arr[i].sort_values(['Tm_Rank', 'Procurement'], inplace = True)
        arr[i].drop('Tm_Rank', 1, inplace = True)

In [ ]:
condition_to_data(nhap_vao)
condition_to_data(xuat_ra)
condition_to_data(thuyen_chuyen)

In [ ]:
thuyen_chuyen[0]

In [ ]:
step_two = pd.concat(nhap_vao + xuat_ra + thuyen_chuyen)


In [ ]:
step_two

In [ ]:
condition_to_delete = ~(step_two['Account code'].isin([101, 102])) & (
    ((step_two['Material Type'] == 'ROH') & (step_two['Procurement'] == 'E')) | 
    ((step_two['Material Type'] == 'HAWA') & (step_two['Procurement'] == 'E')) |
    ((step_two['Material Type'] == 'HALB') & (step_two['Procurement'].isin(['E', 'X']))) | 
    ((step_two['Material Type'] == 'FERT') & (step_two['Procurement'].isin(['E', 'X'])))
)
step_two.drop(step_two[condition_to_delete].index, inplace = True)

In [ ]:
SCM_RAW = step_two

In [ ]:
nhap_vao = [None] * 2
nhap_vao[0] = (
    (step_one['Account code'] == 101) & 
    (step_one['Order Category'] == 'Purchase Order') & 
    (step_one['Material Type'] == 'HALB') & 
    (step_one['Procurement'].isin(['E', 'X']))
)
            
nhap_vao[1] = (
        (step_one['Account code'].isin([602, 623, 701, 720, 801, 809])) &
        (step_one['Material Type'] == 'HALB') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

xuat_ra = [None] * 3
xuat_ra[0] = (
    (step_one['Account code'] == 102) & 
    (step_one['Order Category'] == 'Purchase Order') & 
    (step_one['Material Type'] == 'HALB') & 
    (step_one['Procurement'].isin(['E', 'X']))
)

xuat_ra[1] = (
        (step_one['Account code'].isin([201, 601, 609, 702, 712, 721, 803])) &
        (step_one['Material Type'] == 'HALB') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

xuat_ra[2] = (
        (step_one['Account code'] == 261) &
        (
            ((step_one['Material Type'] == 'HALB') & 
            (step_one['Procurement'].isin(['E', 'X']))) |
            ((step_one['Material Type'] == 'FERT') & 
            (step_one['Procurement'].isin(['E', 'X'])))
        )
)

thuyen_chuyen = [None]
thuyen_chuyen[0] = (
        (step_one['Account code'].between(300, 402)) &
        (step_one['Material Type'] == 'HALB') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

In [ ]:
step_one

In [ ]:
condition_to_data(nhap_vao)
condition_to_data(xuat_ra)
condition_to_data(thuyen_chuyen)

In [ ]:
SCM_WIP = pd.concat(nhap_vao + xuat_ra + thuyen_chuyen)

In [ ]:
SCM_WIP.loc[(
    (SCM_WIP['Account code'] == 261) &
    (SCM_WIP['Material Type'] == 'FERT') &
    (SCM_WIP['Location'].str.startswith('RW'))
), 'Location'] = None

In [ ]:
nhap_vao = [None] * 2
nhap_vao[0] = (
    (step_one['Account code'] == 101) & 
    (step_one['Order Category'] == 'Production Order') & 
    (step_one['Material Type'] == 'FERT') & 
    (step_one['Procurement'].isin(['E', 'X']))
)
            
nhap_vao[1] = (
        (step_one['Account code'].isin([602, 623, 701, 720, 801, 809])) &
        (step_one['Material Type'] == 'FERT') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

xuat_ra = [None] * 2
xuat_ra[0] = (
    (step_one['Account code'] == 102) & 
    (step_one['Order Category'] == 'Production Order') & 
    (step_one['Material Type'] == 'FERT') & 
    (step_one['Procurement'].isin(['E', 'X']))
)

xuat_ra[1] = (
        (step_one['Account code'].isin([201, 601, 609, 702, 712, 721, 803])) &
        (step_one['Material Type'] == 'FERT') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

thuyen_chuyen = [None]
thuyen_chuyen[0] = (
        (step_one['Account code'].between(300, 402)) &
        (step_one['Material Type'] == 'FERT') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

In [ ]:
condition_to_data(nhap_vao)
condition_to_data(xuat_ra)
condition_to_data(thuyen_chuyen)
SCM_FG = pd.concat(nhap_vao + xuat_ra + thuyen_chuyen)

In [ ]:
print('RAW', SCM_RAW.shape)
print('FG', SCM_FG.shape)
print('WIP', SCM_WIP.shape)

In [ ]:
with pd.ExcelWriter('output/before_report.xlsx') as writer:  
    SCM_RAW.to_excel(writer, sheet_name='RAW')
    SCM_FG.to_excel(writer, sheet_name='FG')
    SCM_WIP.to_excel(writer, sheet_name='WIP')

In [ ]:
SCM_REPORT = SCM_RAW.pivot(index=["Unnamed: 0" ,"Material"], columns="Account code", values="Quantity")
SCM_REPORT.reset_index(inplace=True)
del SCM_REPORT['Unnamed: 0']
aggregation_functions = {
    101: sum, 102: 'sum', 
    201: sum, 261: 'sum', 
    343: sum, 401: 'sum',
    601: sum, 609: 'sum',
    720: sum, 721: 'sum',
}
SCM_REPORT = SCM_REPORT.groupby(SCM_REPORT['Material']).aggregate(aggregation_functions)

In [ ]:
col_list = SCM_REPORT.columns.values
col_list

In [ ]:
def get_result(df):
    result = df.pivot(index=["Unnamed: 0" ,"Material"], columns="Account code", values="Quantity")
    result.reset_index(inplace=True)
    del result['Unnamed: 0']
    aggregation_functions = {}
    columns_order = [101, 102, 321, 343, 401, 720, 201, 261, 344, 555, 601, 609, 721]
    for column in result.columns:
        if isinstance(column, int): 
            aggregation_functions[column] = 'sum'
    
    final_col = []
    for col in columns_order:
        if col in result.columns:
            final_col.append(col)
    
    result = result.groupby(result['Material']).aggregate(aggregation_functions)
#     columns_order = ['Material'] + final_col
    result = result[final_col]
    result.reset_index(inplace=True)
    result.index.name = None
    return result

In [ ]:
RAW_REPORT = get_result(SCM_RAW)
WIP_REPORT = get_result(SCM_WIP)
FG_REPORT = get_result(SCM_FG)

In [ ]:
with pd.ExcelWriter('output/final_result.xlsx') as writer:  
    RAW_REPORT.to_excel(writer, sheet_name='REPORT_RAW')
    WIP_REPORT.to_excel(writer, sheet_name='REPORT_FG')
    FG_REPORT.to_excel(writer, sheet_name='REPORT_WIP')

In [ ]:
RAW_REPORT

In [ ]:
score = [None]*65
for num in range(1, 65):
    score[num] = pd.read_excel(f'../data/Score 2021/{num:02}.xlsx')

In [ ]:
score.pop(0)
result = pd.concat(score)

In [ ]:
result

In [ ]:
result.to_excel('total_result.xlsx')

In [ ]:
my_filter = ((result['Văn'].notnull()) & (result['Lịch Sử'].notnull()) & (result['Địa Lý'].notnull()))

In [ ]:
khoi_c = result.loc[my_filter]

In [ ]:
khoi_a['A1'] = khoi_a['Toán'] + khoi_a['Lý'] + khoi_a['Ngoại Ngữ']
khoi_a['A'] = khoi_a['Toán'] + khoi_a['Lý'] + khoi_a['Hoá']

In [ ]:
khoi_a['TONG'] = khoi_a[["A1", "A"]].max(axis=1)

In [ ]:
# khoi_a.reset_index(inplace=True)

In [ ]:
khoi_a.sort_values(by='TONG', ascending=False, inplace=True)

In [ ]:
khoi_a.reset_index()

In [ ]:
khoi_a.to_excel('khoia.xlsx')

In [ ]:
khoi_a

In [ ]:
khoi_c['C'] = khoi_c['Văn'] + khoi_c['Lịch Sử'] + khoi_c['Địa Lý']

In [ ]:
khoi_c

In [ ]:
khoi_c.sort_values(by='C', ascending=False, inplace=True)

In [ ]:
khoi_c

In [ ]:
khoi_c.to_excel('khoi_c.xlsx')